In [138]:
import pandas as pd
from glob import glob
import gsw
import numpy as np
from geopy.distance import distance

df2012 = pd.read_csv('../../data/acousticData/2012_2013/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2012_v2.csv')
df2013 = pd.read_csv('../../data/acousticData/2012_2013/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2013_v2.csv')
df2012Acoustics = df2012.iloc[:, [0,1,2,3,4]]
df2013Acoustics = df2013.iloc[:, [0,1,2,3,4]]
df2018Acoustics = pd.read_csv('../../data/acousticData/2017_2019/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2017.csv')
df2017Acoustics = pd.read_csv('../../data/acousticData/2018/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2018.csv')
df2017Acoustics = df2017Acoustics[df2017Acoustics.Latitude!=999]
df2019Acoustics = pd.read_csv('../../data/acousticData/2017_2019/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2019.csv')
df2019Acoustics = df2019Acoustics[df2019Acoustics.Latitude!=999]

df2012Catch = df2012.iloc[:, [0,1,2,5,6,7,8]]
df2013Catch = df2013.iloc[:, [0,1,2,5,6,7,8]]
df2017Catch = pd.read_csv('../../data/catchData/2017_2019/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2017.csv')
df2017Catch = df2017Catch[df2017Catch.Latitude!=999]
df2019Catch = pd.read_csv('../../data/catchData/2017_2019/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2019.csv')
df2019Catch = df2019Catch[df2019Catch.Latitude!=999]

dfCTD = pd.concat([pd.read_csv(file) for file in glob('../../data/physicalData/*.csv')])
dfCTD = dfCTD[dfCTD['profile_id'].notna()]
dfCTD = dfCTD.drop(labels=['S_42'],axis=1)
dfCTD = dfCTD.astype({'pressure': float, 'latitude': float, 'longitude': float, 'S_41': float, 'T2_35': float,'T_28': float})
dfCTD['time'] = pd.to_datetime(dfCTD.time)
dfCTD['depth'] = -1*gsw.z_from_p(dfCTD.pressure, dfCTD.latitude)
dfCTD = dfCTD.sort_values(by='time')

In [141]:
df2019Catch

,Interval,Latitude,Longitude,Arctic cod (No. m^-2),Pollock (No. m^-2),Saffron cod (No. m^-2),Capelin (No. m^-2),Pacific cod (No. m^-2)
0,20190827204410,72.500002,-168.493824,0.080165,0.000000,0.006533,0.0,0.000000
1,20190827204752,72.499674,-168.465049,0.314305,0.000000,0.025616,0.0,0.000000
2,20190827205130,72.499930,-168.437338,0.546501,0.000000,0.044539,0.0,0.000000
3,20190827205451,72.500105,-168.409674,0.710665,0.000000,0.057919,0.0,0.000000
4,20190827205811,72.500257,-168.382095,0.714433,0.000000,0.058226,0.0,0.000000
...,...,...,...,...,...,...,...,...
2395,20190926165700,66.500355,-168.427843,0.000000,0.004750,0.004932,0.0,0.000028
2396,20190926170021,66.500280,-168.448750,0.000000,0.005047,0.005240,0.0,0.000030
2397,20190926170341,66.500121,-168.469629,0.000000,0.012387,0.012862,0.0,0.000073
2398,20190926170700,66.500045,-168.490529,0.000000,0.001106,0.001148,0.0,0.000007


In [114]:
def ctdStationRadius(df):
    dfStations = df[['profile_id','latitude','longitude','time']].drop_duplicates()
    dists = []
    for year in df.time.dt.year.unique():
        dfCur = dfStations[dfStations.time.dt.year == year]
        nPoints = len(dfCur.latitude)
        for i in range(nPoints):
            curDists = []
            for ii in range(nPoints):
                if ii == i:
                    continue
                curDists.append(distance((dfCur.latitude.values[i],dfCur.longitude.values[i]),(dfCur.latitude.values[ii],dfCur.longitude.values[ii])).nm)
            dists.append(np.min(curDists))
    dfStations['TrackRadius'] = np.array(dists)/2
    
    S_max, S_min, S_surf, S_bot, T_min, T_max, T_surf, T_bot = [[] for i in range(8)]

    for pid in df.profile_id.unique():
        S_max.append(df[(df.profile_id==pid)].S_41.max())
        S_min.append(df[(df.profile_id==pid)].S_41.min())
        S_surf.append(df[(df.profile_id==pid)&(df.depth < 5)].S_41.mean())
        S_bot.append(df[(df.profile_id==pid)&(df.depth > df[(df.profile_id==pid)].depth.max()-5)].S_41.mean())
        T_max.append(df[(df.profile_id==pid)].T_28.max())
        T_min.append(df[(df.profile_id==pid)].T_28.min())
        T_surf.append(df[(df.profile_id==pid)&(df.depth < 5)].T_28.mean())
        T_bot.append(df[(df.profile_id==pid)&(df.depth > df[(df.profile_id==pid)].depth.max()-5)].T_28.mean())
    dfStats = pd.DataFrame({'profile_id':df.profile_id.unique(),'S_max':S_max, 'S_min':S_min, 'S_surf':S_surf, 'S_bot':S_bot, 'T_min':T_min, 'T_max':T_max, 'T_surf':T_surf, 'T_bot':T_bot})
    return dfStations.merge(dfStats)

In [119]:
dfStations.merge(dfStats)

,profile_id,latitude,longitude,time,TrackRadius,S_max,S_min,S_surf,S_bot,T_min,T_max,T_surf,T_bot
0,os1701l1c001_ctd,67.910835,191.81500,2017-08-06 15:42:36+00:00,0.354903,32.2600,31.3250,31.325000,32.248283,3.7013,7.8523,7.847600,3.810117
1,os1701l1c002_ctd,70.211670,192.21000,2017-08-07 08:36:51+00:00,3.789215,32.7450,32.1094,32.109400,32.744667,2.1981,8.4366,8.436600,3.390350
2,os1701l1c003_ctd,70.017500,193.15016,2017-08-07 23:26:35+00:00,0.034635,32.4606,31.9856,31.985600,32.453450,4.0813,8.8102,8.810200,4.090983
3,os1701l1c004_ctd,70.018500,193.14850,2017-08-08 00:00:45+00:00,0.034635,32.4623,31.9401,31.940100,32.454550,4.0746,9.0188,9.018800,4.088733
4,os1701l1c005_ctd,71.233330,195.77834,2017-08-08 15:17:49+00:00,0.164573,32.7043,31.7970,31.797100,32.686400,-0.9993,6.1166,6.116600,2.068417
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,os1901l3c087_ctd,67.250660,195.28767,2019-09-23 21:42:27+00:00,8.898219,31.3532,29.9941,29.999383,31.193083,8.5742,9.9187,9.580383,8.974450
215,os1901l3c088_ctd,66.873665,195.00116,2019-09-24 04:37:37+00:00,11.839213,30.0192,29.7265,29.726883,29.929400,10.1218,10.2735,10.125517,10.244467
216,os1901l3c089_ctd,66.498000,193.97800,2019-09-24 16:28:54+00:00,14.728148,29.2437,29.2429,29.243617,29.243320,9.8508,9.8661,9.861567,9.863640
217,os1901l3c090_ctd,66.501335,192.75250,2019-09-24 20:18:43+00:00,14.728148,29.5534,27.6054,27.605600,29.141567,9.8944,10.7004,9.897350,10.660567


In [ ]:
from tqdm import tqdm_notebook as tqdm
#dfStat = ctdStationRadius(dfCTD)
meanSa, meanAcod, meanScod, meanPcod, meanPol = ([] for i in range(5))
for  year in dfStat.time.dt.year.unique():
    dfEnvCur = dfStat[dfStat.time.dt.year == year]
    if year == 2017:
        dfFish = df2017Catch
        dfAcoustics = df2017Acoustics
    else:
        dfFish = df2019Catch
        dfAcoustics = df2019Acoustics
    for i in tqdm(range(len(dfEnvCur))):
        curSa, curAcod, curScod, curPcod, curPol = ([] for i in range(5))
        curSite = (dfEnvCur.latitude.values[i],dfEnvCur.longitude.values[i])
        curRad = dfEnvCur.TrackRadius.values[i]
        for ii in range(len(dfFish)):
            if distance(curSite,(dfFish.Latitude.values[ii],dfFish.Longitude.values[ii])).nm <= curRad:
                curAcod.append(dfFish[dfFish.columns[3]].values[ii])
                curScod.append(dfFish[dfFish.columns[5]].values[ii])
                curPcod.append(dfFish[dfFish.columns[4]].values[ii])
                curPol.append(dfFish[dfFish.columns[6]].values[ii])
        for ii in range(len(dfAcoustics)):
            if distance(curSite,(dfAcoustics.Latitude.values[ii],dfAcoustics.Longitude.values[ii])).nm <= curRad:
                curSa.append(dfAcoustics[dfAcoustics.columns[4]].values[ii])
        meanSa.append(np.mean(curSa))
        meanAcod.append(np.mean(curAcod))
        meanScod.append(np.mean(curScod))
        meanPcod.append(np.mean(curPcod))
        meanPol.append(np.mean(curPol))
dfStat['meanSa'] = meanSa
dfStat['meanAcod'] = meanAcod
dfStat['meanScod'] = meanScod
dfStat['meanPcod'] = meanPcod
dfStat['meanPol'] = meanPol
dfStat.head()

<ipython-input-142-9113a49f3448>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(dfEnvCur))):


C:\Users\rober\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rober\Anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
